# Taller 1. Introducción a R. Estadística descriptiva de rasgos continuo
Genética de rasgos complejos 2024-01

__Hans D. Escobar H.__

1. Escriba un código de mínimo dos líneas para la varianza fenotípica total (Vp) para las 
localidades de Subachoque y Sumapaz.
 
2. Escriba un código de mínimo dos líneas para estimar la heredabilidad en sentido amplio 
para cada localidad.

In [1]:
# Environment setup

# Required libraries validation
if (!require("lme4", quietly = TRUE)) {
  print("Error: missing lme4 package")
  quit()
}

# V_P and H^2 functions
phenotypic_variance <- function(line_variance, residual_variance) {
  # V_P = V_G + V_E , V_G  := line_variance, V_E := residual_variance
  return(line_variance + residual_variance)
}

broad_sense_heritability <- function(line_variance, phenotypic_variance) {
  # $H^2 = \frac{V_G}{V_P}$ - additive and non-additive genetic effects
  return(line_variance / phenotypic_variance)
}


In [2]:
# Generate pelargonina table

get_pelargonina_variances <- function(location_df) {
  # Fit a linear mixed-effects model (LMM) to data
  lmer_mod_object <- lmer(Pelargonina ~ (1 | Line), data = location_df)
  # Extract Variance and Correlation Components
  vc_df <- as.data.frame(VarCorr(lmer_mod_object))[, c("grp", "vcov")]
  return(vc_df)
}


data_df <- read.table("Example_Data/pelargo.txt", header = TRUE)

cat("Raw data head (size=", nrow(data_df), "):", sep = "")
head(data_df)

pelargonina_summary <- data.frame(
  # subachoque 1, sumapaz 2
  "Location" = factor(levels = c("subachoque", "sumapaz")),
  "V_G" = double(),
  "V_E" = double(),
  "V_P" = double(),
  "H^2" = double()
)

for (loc in c("1", "2")) {
  # Generate variance values
  variances_df <- get_pelargonina_variances(data_df[data_df$Location == loc, ])
  line_variance <- variances_df[variances_df$grp == "Line", ][, "vcov"]
  residual_variance <- variances_df[variances_df$grp == "Residual", ][, "vcov"]
  v_p <- phenotypic_variance(line_variance, residual_variance)
  broad_heritability <- broad_sense_heritability(line_variance, v_p)

  # Validation of location values
  if (loc == 1) {
    location <- "subachoque"
  } else if (loc == 2) {
    location <- "sumapaz"
  } else {
    throw("Invalid location value", loc)
  }

  # Generate summary data frame
  pelargonina_summary[nrow(pelargonina_summary) + 1, ] <- list(
    location, line_variance, residual_variance, v_p, broad_heritability
  )
}


Raw data head (size=60):

,Location,Line,Pelargonina
,<int>,<chr>,<dbl>
1,1,a,12.43
2,1,a,12.42
3,1,a,12.40
4,2,a,11.23
5,2,a,11.22
6,2,a,11.60


In [3]:
pelargonina_summary


,Location,V_G,V_E,V_P,H.2
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,subachoque,134.35622,0.28271,134.63893,0.9979002
2,sumapaz,69.48284,0.27114,69.75398,0.9961129


3. ¿Qué puede concluir de los resultados?

El análisis realizado falla en capturar el efecto ambiental. Dado a que son las mismas líneas en distintos ambientes, se espera variabilidad genética similar; sin embargo, en Subachoque fue de casi el doble que en Sumapaz. Es necesario incluir comparaciones de la misma línea en distintas locaciones.